In [1]:
%autosave 0

Autosave disabled


In [2]:
!pip freeze | grep tensorflow

tensorflow==2.8.2
tensorflow-addons==0.17.1
tensorflow-estimator==2.8.0
tensorflow-io-gcs-filesystem==0.26.0
tensorflow-serving-api==2.7.0


In [3]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0 -q

In [4]:
!pip install keras-image-helper -q

In [5]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [6]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [7]:
from keras_image_helper import create_preprocessor

In [8]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [9]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [10]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [11]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [12]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [19]:
preds = pb_response.outputs['dense_7'].float_val
preds

[-1.87986421585083, -4.75631046295166, -2.359531879425049, -1.08926522731781, 9.903782844543457, -2.826179027557373, -3.648310422897339, 3.241154909133911, -2.612096071243286, -4.852035045623779]

In [20]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds))

{'dress': -1.87986421585083,
 'hat': -4.75631046295166,
 'longsleeve': -2.359531879425049,
 'outwear': -1.08926522731781,
 'pants': 9.903782844543457,
 'shirt': -2.826179027557373,
 'shoes': -3.648310422897339,
 'shorts': 3.241154909133911,
 'skirt': -2.612096071243286,
 't-shirt': -4.852035045623779}